# 5章 顧客の退会を予測する

引き続き、スポーツジムの会員データを使って顧客の行動を分析していきます。  
３章では顧客の全体像を把握し、4章では数ヶ月利用している顧客の来月の利用回数の予測を行いました。   
ここでは、教師あり学習の分類を用いて、顧客の退会予測を取り扱います。

In [29]:
import pandas as pd

customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')

### ノック41：データを読み込んで利用データを整形しよう

In [30]:
# 当月とその1ヶ月前のデータを使用するため、データを絞り込む。
year_months = list(uselog_months['年月'].unique())
# 絞ったデータを格納するdf
uselog = pd.DataFrame()

# 当月とその一月前のデータを取得するから、rangeは1からスタート
for i in range(1, len(year_months)):
    # 当月の利用
    tmp = uselog_months.loc[uselog_months['年月']==year_months[i]].copy()
    tmp.rename(columns={'count':'count_0'}, inplace=True)
    # 当月から1ヶ月前の利用
    tmp_before = uselog_months.loc[uselog_months['年月']==year_months[i-1]].copy()
    del tmp_before['年月']
    tmp_before.rename(columns={'count':'count_1'}, inplace=True)
    # uselogに結合する
    tmp = pd.merge(tmp, tmp_before, on='customer_id', how='left')
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


### ノック42：退会前月の退会顧客データを作成しよう

In [42]:
# 退会を未然に防ぎたいため、退会申請を出す前の月のデータから退会申請を出す確率を予測する
from dateutil.relativedelta import relativedelta

# 退会済み会員のみに絞る
exit_customer = customer.loc[customer['is_deleted']==1].copy()
# indexをリセットする
exit_customer.reset_index(drop=True, inplace=True)
exit_customer['exit_date'] = None
# datetime型に変換
exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])

In [43]:
for i in range(len(exit_customer)):
    exit_customer.loc[i, 'exit_date'] = exit_customer.loc[i, 'end_date'] - relativedelta(months=-1)

In [44]:
print(exit_customer['exit_date'].dtypes)

object


In [45]:
exit_customer['exit_date'] = pd.to_datetime(exit_customer['exit_date'])

In [35]:
exit_customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-04-30,23,2018-05-30
1,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,デイタイム,7500,通常,3.0,3.0,3,3,0,2018-04-30,23,2018-05-30
2,HD542886,XX,C01,M,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,1.0,1.0,1,1,0,2018-04-30,23,2018-05-30
3,HD597545,XXXXX,C03,F,2016-06-01,2018-05-31,CA1,1,ナイト,6000,通常,3.5,3.5,4,3,1,2018-05-31,23,2018-06-30
4,HI749296,XXXXX,C01,M,2016-06-01,2018-05-31,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-05-31,23,2018-06-30


In [66]:
# 利用履歴データと退会会員データをマージする
exit_customer['年月'] = exit_customer['exit_date'].dt.strftime('%Y%m').astype(str)
uselog['年月'] = uselog['年月'].astype(str)
print(list(uselog['年月'].unique()))
print(list(exit_customer['年月'].unique()))

# データ型の確認
print(exit_customer['年月'].dtypes)
print(uselog['年月'].dtypes)

['201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903']
['201805', '201806', '201807', '201808', '201809', '201810', '201812', '201901', '201902', '201811', '201903', '201904']
object
object


In [74]:
exit_uselog = pd.merge(uselog, exit_customer, on=['customer_id', '年月'], how='left')

print(len(exit_uselog))
exit_uselog.head()

33851


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [76]:
# 欠損値を削除。name列に欠損値がある行だけを削除
exit_uselog = exit_uselog.dropna(subset=['name'])
print(len(exit_uselog))
print(len(exit_uselog['customer_id'].unique()))
exit_uselog.head()

0
0


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date


In [ ]:
# 退会顧客を軸に集計した退会前月の顧客データの完成

### ノック43：継続顧客のデータを作成しよう

In [89]:
customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,ナイト,6000,通常,3.916667,4.0,6,1,1,2019-04-30,47


In [90]:
uselog.head()

,年月,customer_id,count_0,count_1
0,201903,AS002855,4,5.0
1,201903,AS008805,1,6.0
2,201903,AS009373,5,6.0
3,201903,AS015233,8,4.0
4,201903,AS015315,4,5.0


In [91]:
conti_customer = customer.loc[customer['is_deleted'] == 0].copy()
conti_customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,ナイト,6000,通常,3.916667,4.0,6,1,1,2019-04-30,47


In [92]:
conti_customer = pd.merge(uselog, conti_customer, on='customer_id', how='left')
conti_customer

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201903,AS002855,4,5.0,XXXX,C03,F,2016-11-01,NaN,CA1,...,ナイト,6000.0,通常,4.500000,5.0,7.0,2.0,1.0,2019-04-30,29.0
1,201903,AS008805,1,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201903,AS009373,5,6.0,XX,C01,F,2015-11-01,NaN,CA1,...,オールタイム,10500.0,通常,5.083333,5.0,7.0,3.0,1.0,2019-04-30,41.0
3,201903,AS015233,8,4.0,XXXXX,C01,M,2018-05-13,NaN,CA2,...,オールタイム,10500.0,入会費半額,7.545455,7.0,11.0,4.0,1.0,2019-04-30,11.0
4,201903,AS015315,4,5.0,XXXXX,C01,M,2015-07-01,NaN,CA1,...,オールタイム,10500.0,通常,4.833333,5.0,7.0,3.0,1.0,2019-04-30,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2948,201903,TS995853,8,11.0,XXXX,C01,M,2019-02-08,NaN,CA1,...,オールタイム,10500.0,通常,9.500000,9.5,11.0,8.0,1.0,2019-04-30,2.0
2949,201903,TS998593,8,7.0,XXXXX,C03,M,2018-09-01,NaN,CA1,...,ナイト,6000.0,通常,8.142857,8.0,9.0,7.0,1.0,2019-04-30,7.0
2950,201903,TS999079,3,2.0,XXX,C03,M,2016-06-01,NaN,CA1,...,ナイト,6000.0,通常,4.916667,5.5,9.0,2.0,1.0,2019-04-30,34.0
2951,201903,TS999231,6,6.0,XXXX,C01,M,2017-03-01,NaN,CA1,...,オールタイム,10500.0,通常,4.666667,5.0,8.0,1.0,1.0,2019-04-30,25.0


In [93]:
conti_customer.count()

年月                   2953
customer_id          2953
count_0              2953
count_1              2888
name                 2842
class                2842
gender               2842
start_date           2842
end_date                0
campaign_id          2842
is_deleted           2842
class_name           2842
price                2842
campaign_name        2842
mean                 2842
median               2842
max                  2842
min                  2842
routine_flg          2842
calc_date            2842
membership_period    2842
dtype: int64

In [116]:
conti_customer = conti_customer.dropna(subset=['name'])
len(conti_customer)

2842

### ノック44：予測する月の在籍期間を作成しよう

### ノック45：欠損値を除去しよう

### ノック46：文字列型の変数を処理できるように整形しよう

### ノック47：決定木を用いて退会予測モデルを作成してみよう

### ノック48：予測モデルの評価を行ない、モデルのチューニングをしてみよう

### ノック49：モデルに寄与している変数を確認しよう

### ノック50：顧客の退会を予測しよう